# Loading Data

In [1]:
import pickle as pkl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

fileName =  '../input/a1-data-to-similar-events/Similarity'
fileObject2 = open(fileName, 'rb')
Imported_data = pkl.load(fileObject2)
fileObject2.close()

#print('Similarity Thresholds:',Imported_data[:,0])

#print('Most Similarity at:',Imported_data[:,1])

Similar_event_count = len(Imported_data[0,:])-1
print('Number of Similar Samples Considered', Similar_event_count)

Similarity_thresholds = Imported_data[:,0]
for iter1 in range(len(Similarity_thresholds)):
    if Similarity_thresholds[iter1] < 1e15:
        continue
    Similarity_thresholds[iter1] = 1e15 # Avoiding NaN values 
    
Sample_density = Similar_event_count/ Similarity_thresholds/ 100
#Sample_density = number of samples with less than 1% (of range) deviation

Similar_samples = Imported_data[:,1:Similar_event_count+1]

print('Target Shape', Sample_density.shape, 'Similar Samples Matrix Shape:', Similar_samples.shape)

Number of Similar Samples Considered 100
Target Shape (20000,) Similar Samples Matrix Shape: (20000, 100)


In [2]:
train_csv_path = '../input/electricity-uk-grid-data-preparation/Wind_train.csv'
test_csv_path = '../input/electricity-uk-grid-data-preparation/Wind_test.csv'
val_csv_path = '../input/electricity-uk-grid-data-preparation/Wind_val.csv'



import pandas as pd
train_df = pd.read_csv(train_csv_path)
val_df = pd.read_csv(val_csv_path)
train_df.head()

,y(t-4),y(t-3),y(t-2),y(t-1),Hour,Day,y(t)
0,1529.0,1561.0,1573.0,1559.0,16.835556,177.0,1565.0
1,1561.0,1573.0,1559.0,1565.0,16.916944,177.0,1587.0
2,1684.0,1695.0,1750.0,1800.0,17.666944,177.0,1838.0
3,1695.0,1750.0,1800.0,1838.0,17.750556,177.0,1836.0
4,1812.0,1805.0,1769.0,1758.0,18.500278,177.0,1773.0


# Training-Validation Splitting
The training Dataset is split, as top matches and sample densities are computed on the training set.

In [3]:
import torch
from torch.autograd import Variable
import torch.nn.functional as F
import torch.utils.data as Data
import imageio

input_=[]
output_=[]
for row in train_df.iloc:
    input_.append((row[0:len(row)-1]).astype(float))
    output_.append(row[-1])
    
    
num_input = len(row)-1;
print(num_input)

torch.manual_seed(1)    # reproducible

x = torch.tensor(input_[0:4000]).float()  
print('Shape of training input:',x.shape)
y = torch.tensor(Sample_density[0:4000]).float()   
new_shape = (len(y), 1)
y = y.view(new_shape)


i_val = torch.tensor(input_[4000:5000]).float()  
print('Shape of validation input:',i_val.shape)
o_val = torch.tensor(Sample_density[4000:5000]).float()   
new_shape = (len(o_val), 1)
o_val = o_val.view(new_shape)



max_y = torch.max(y[:,0])
min_y =torch.min(y[:,0])

max_x = torch.max(x,dim=0)
min_x = torch.min(x,dim=0)

print(max_y, min_y, max_x.values, min_x.values)

range_y = max_y - min_y
range_x = max_x.values - min_x.values

print(range_x, range_y)

    #Normalizing
x = (x - min_x.values)/range_x
y = (y - min_y)/range_y

i_val = (i_val - min_x.values)/range_x
o_val = (o_val - min_y)/range_y


6
Shape of training input: torch.Size([4000, 6])
Shape of validation input: torch.Size([1000, 6])
tensor(3363934.) tensor(1.0000e-15) tensor([2843.0000, 2848.0000, 2848.0000, 2830.0000,   23.9206,  245.0000]) tensor([2.1000e+01, 2.2000e+01, 2.2000e+01, 2.2000e+01, 2.7778e-04, 1.7700e+02])
tensor([2822.0000, 2826.0000, 2826.0000, 2808.0000,   23.9203,   68.0000]) tensor(3363934.)


# Sample Density NN Training

In [4]:
# torch can only train on Variable, so convert them to Variable
x, y = Variable(x), Variable(y)
 
class Net(torch.nn.Module):
    def __init__(self, n_feature, n_hidden, n_output):
        super(Net, self).__init__()
        self.hidden = torch.nn.Linear(n_feature, n_hidden)   # hidden layer
        self.hidden2 = torch.nn.Linear(n_hidden, n_hidden)   # hidden layer
        self.predict = torch.nn.Linear(n_hidden, n_output)   # output layer

    def forward(self, x):
        x = F.relu(self.hidden(x))      # activation function for hidden layer
        x = F.relu(self.hidden2(x))
        x = self.predict(x)             # linear output
        return x

# use the same net as before      
net = Net(n_feature=num_input, n_hidden=500, n_output=1)     # define the network
print(net)  # net architecture
optimizer = torch.optim.Adam(net.parameters(), lr=0.05)
loss_func = torch.nn.MSELoss()  # this is for regression mean squared loss


minimum_train_loss = 1e5
minimum_val_loss = 1e5
EPOCH = 600

# start training
for epoch in range(EPOCH):
  
    prediction = net(x)     # input x and predict based on x

    loss = loss_func(prediction, y)     # must be (1. nn output, 2. target)

    optimizer.zero_grad()   # clear gradients for next train
    loss.backward()         # backpropagation, compute gradients
    optimizer.step()        # apply gradients
    
    if loss<minimum_train_loss:
        minimum_train_loss =loss
        net_opt = net
    
    if epoch%200 == 199:
      prediction = net_opt(i_val)
      loss_val = loss_func(prediction, o_val)
      if loss_val<minimum_val_loss:
        minimum_val_loss = loss_val
        net_opt_val = net_opt
      print ("Epoch [{}/{}], Loss: {:.4f}, Minimum Loss {:.6f}, Val Loss {:.6f}  "  .format(epoch+1, EPOCH, loss, minimum_train_loss, minimum_val_loss))

Net(
  (hidden): Linear(in_features=6, out_features=500, bias=True)
  (hidden2): Linear(in_features=500, out_features=500, bias=True)
  (predict): Linear(in_features=500, out_features=1, bias=True)
)
Epoch [200/600], Loss: 0.0101, Minimum Loss 0.010078, Val Loss 0.016552  
Epoch [400/600], Loss: 0.0082, Minimum Loss 0.008163, Val Loss 0.016552  
Epoch [600/600], Loss: 0.0078, Minimum Loss 0.007789, Val Loss 0.016552  
